In [39]:
import aiohttp
import asyncio
import json
import requests
import webbrowser

from pathlib import Path
from typing import Dict, Any

In [7]:
MEMES_PATH = Path('./new_memes.json')
USERNAME = 'memegpt'
PASSWORD = ''

In [3]:
with open(MEMES_PATH) as f:
    memes = json.load(f)

In [32]:
def get_params(meme):
    params = {
        "template_id": meme['id'],
        'username': USERNAME,
        'password': PASSWORD,
    }
    if 'boxes' in meme:
        boxes = meme['boxes']
    else:
        boxes = ['', '', '']
    for i, text in enumerate(boxes):
        params[f'boxes[{i}][text]'] = f'text{i}'
    return params

In [22]:
async def async_generate_meme(session, params: Dict[str, Any]):
    async with session.post('https://api.imgflip.com/caption_image', data=params, ssl=False) as response:
        return await response.json()

In [38]:
async def get_meme_urls():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for meme in memes['memes']:
            tasks.append(asyncio.ensure_future(async_generate_meme(session, get_params(meme))))
        meme_responses = await asyncio.gather(*tasks)
        return [response_json['data']['url'] for response_json in meme_responses]

In [40]:
def open_meme_urls(meme_urls):
    for url in meme_urls:
        webbrowser.open(url)

In [41]:
meme_urls = await get_meme_urls()

In [42]:
open_meme_urls(meme_urls)